In [1]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import sys

class SensorData:
    def __init__(self,sensor_id):
        self.id = sensor_id
        self.directions = []
        self.real_traff = []
        self.simul_traff = []
        
    def add_direction(self,direction,real,simul):
        self.directions.append(direction)
        self.real_traff.append(real)
        self.simul_traff.append(simul)
	
map = 'test_map'
from_date = '2020-02-18'
to_date = '2020-02-19'
print(from_date)
print(to_date)
	
simul_period = from_date.replace('-','') + 'T0000_' + to_date.replace('-','') + 'T0000'
print(simul_period)

sensor_loc = pd.read_csv('../sensors/sensor_location.csv',delimiter=';')
data_simul = pd.read_csv('../outputs/'+ map + '/inductiondetections.csv', delimiter=',', engine='python')

#sensors_in_map =[]
#for s in data_simul['Detector'].unique():
#    sensors_in_map.append(s.split(sep='_')[0])
sensors_in_map=data_simul['Detector'].str.split(pat='_',n=1).str[0].unique()

#Start by creating a dict to index all sensors in map
all_sensor = {}
#Iterate over all sensors
for sens in sensors_in_map:
    print(sens)
    #For each sensor add a new entry in the dict
    all_sensor[sens]=SensorData(sens)
    #And read real traffic data (sensor id helps to get the correct file)
    data_real=pd.read_csv('../sensors/data/'+sens+'_hour_'+ simul_period +'.csv', delimiter=';', engine='python', na_values=['-'])
    
    #Then, for each sensor, loop through both directions. Final goal is to sum up the traffic in both directions
    for direct in sensor_loc[sensor_loc['Detector ID']==sens]['Direction'].unique():
        print(direct)
        #Get all lanes in given direction
        lanes = sensor_loc[(sensor_loc['Detector ID']==sens) & (sensor_loc['Direction']==direct)]['Detector Lane']
        #Initialize array for traffic in this direction (number of entries equal to number of hours in the csv)
        n_hours = len(data_real[data_real['Felt']=='Totalt'])
        real_traff_direction = np.zeros(n_hours)
        simul_traff_direction = np.zeros(n_hours)
        #Loop over all lanes in this direction, and sum traffic ammount on lane to direction total
        for lane in lanes:
            print(lane)
            #Read real traffic data for lane and add it to direction count
            #print('new_val = {}'.format(data_real[data_real['Felt']==str(lane)]['Volum'].values))
            real_traff_direction += data_real[data_real['Felt']==str(lane)]['Volum'].values
            print('real_traff_direction = {}'.format(real_traff_direction))
            
            #Now read simulated data, and parse it according to its format (is not yet aggregated per hour)
            simul_data_lane = data_simul[data_simul.Detector == str(sens+'_'+str(lane))]
            h=1
            simul_traff_lane = np.zeros(n_hours)
            for index, row in simul_data_lane.iterrows():
                if row.Time > n_hours*3600:
                    continue
                if row.Time <= h*3600:
                    simul_traff_lane[h-1] += row.qPKW
                else:
                    h+=1
                    simul_traff_lane[h-1] += row.qPKW
            
            simul_traff_direction += simul_traff_lane
            print('simul_traff_lane = {}'.format(simul_traff_lane))
            
        #Save result in data structure
        #print(direct)
        #print(all_sensor[sens].directions)
        all_sensor[sens].add_direction(direction=direct,real=real_traff_direction,simul=simul_traff_direction)
        
        
if not os.path.exists('figs/from_'+ from_date + '_to_' + to_date):
        os.mkdir('figs/from_'+ from_date + '_to_' + to_date)
    
for sens in all_sensor:
    for idx, direct in enumerate(all_sensor[sens].directions):
        plt.plot(all_sensor[sens].real_traff[idx],label='real traffic')
        plt.plot(all_sensor[sens].simul_traff[idx],label='simul traffic')
        plt.xlabel("Hour")
        plt.ylabel("Traffic")
        plt.legend()
        plt.savefig('figs/from_'+ from_date + '_to_' + to_date + '/'+str(sens)+'_to_'+str(direct)+'.png',format='png',dpi=1200)
        plt.clf()


2020-02-18
2020-02-19
20200218T0000_20200219T0000
21571V2394246
Nyhavna
1
real_traff_direction = [  3.   7.   5.   1.   1.  10.  96. 475. 312. 116.  86. 110. 125.  97.
 119. 215. 190. 122. 110.  93.  41.  38.  25.  14.]
simul_traff_lane = [  3.   8.   6.   1.   1.  12. 116. 569. 376. 133.  98. 123. 155.  64.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
3
real_traff_direction = [  31.   27.   17.   15.   40.   88.  506. 1295. 1008.  653.  607.  645.
  719.  632.  703.  928.  883.  705.  633.  571.  394.  265.  181.  104.]
simul_traff_lane = [  0.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
   1.   1. 210.   1.   1.   1.   1.   1.   3.  13.]
04211V1677432
Flakk
1
real_traff_direction = [ 13.   5.   8.   7.  19.  24.  88. 156. 158. 122. 111. 108. 123. 144.
 190. 362. 250. 196. 195. 174. 115.  76.  53.  34.]
simul_traff_lane = [ 15.  11.   4.   3.  18.  24. 131. 201. 153. 164. 139. 166. 172.   1.
 285. 559.   4.   1.   1. 244. 214. 120.  83.  25.]
Ila
2
real_tra

simul_traff_lane = [ 53.  33.  29.  20.  27.  72. 233. 654. 498. 238. 247.  97.   1.   1.
  81.  27.  16.  77.   2.  87.  11.   0.   0.   0.]
3
real_traff_direction = [  50.   25.   25.   20.   40.   72.  346.  744.  710.  581.  586.  621.
  688.  777.  857. 1421. 1239.  750.  670.  512.  500.  403.  248.  151.]
simul_traff_lane = [  0.   1.   1.   1.   1.   1.  34. 205. 166.  46.  45.   8.   1.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
44656V72812
Sentrum
1
real_traff_direction = [ 52.  32.  20.  25.  26.  35. 178. 554. 484. 471. 578. 574. 653. 724.
 839. 924. 687. 482. 415. 407. 408. 345. 296. 156.]
simul_traff_lane = [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
3
real_traff_direction = [  73.   35.   25.   33.   29.   42.  213.  634.  593.  549.  662.  660.
  754.  843. 1001. 1193.  852.  578.  483.  469.  460.  390.  346.  187.]
simul_traff_lane = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Heimdal
2
real_tra

<Figure size 432x288 with 0 Axes>